## Naive Bayes

In [1]:
from typing import Set
import re

In [3]:
def tokenize(text: str) -> Set[str]:
    text = text.lower()
    all_words = re.findall("[a-z0-9']+", text)
    return set(all_words)

assert tokenize('Data science is science') == {"data", 'science', 'is'}

In [4]:
from typing import NamedTuple

class Message(NamedTuple):
    text: str
    is_spam: bool

In [24]:
from typing import List, Dict, Tuple, Iterable
import math
from collections import defaultdict

class NaiveBayesClassifier:
    def __init__(self, k: float = 0.5) -> None:
        self.k = k  # smoothing factor

        self.tokens: Set[str] = set()
        self.token_spam_counts: Dict[str, int] = defaultdict(int)
        self.token_ham_counts: Dict[str, int] = defaultdict(int)
        self.spam_messages = self.ham_messages = 0

    def train(self, messages: Iterable[Message]) -> None:
        for message in messages:
            # Increment message counts
            if message.is_spam:
                self.spam_messages += 1
            else:
                self.ham_messages += 1

            # Increment word counts
            for token in tokenize(message.text):
                self.tokens.add(token)
                if message.is_spam:
                    self.token_spam_counts[token] += 1
                else:
                    self.token_ham_counts[token] += 1

    def _probabilities(self, token: str) -> Tuple[float, float]:
        """returns P(token | spam) and P(token | not spam)"""
        spam = self.token_spam_counts[token]
        ham = self.token_ham_counts[token]

        p_token_spam = (spam + self.k) / (self.spam_messages + 2 * self.k)
        p_token_ham = (ham + self.k) / (self.ham_messages + 2 * self.k)

        return p_token_spam, p_token_ham

    def predict(self, text: str) -> float:
        text_tokens = tokenize(text)
        log_prob_if_spam = log_prob_if_ham = 0.0

        # Iterate through each word in our vocabulary.
        for token in self.tokens:
            prob_if_spam, prob_if_ham = self._probabilities(token)

            # If *token* appears in the message,
            # add the log probability of seeing it;
            if token in text_tokens:
                log_prob_if_spam += math.log(prob_if_spam)
                log_prob_if_ham += math.log(prob_if_ham)

            # otherwise add the log probability of _not_ seeing it
            # which is log(1 - probability of seeing it)
            else:
                log_prob_if_spam += math.log(1.0 - prob_if_spam)
                log_prob_if_ham += math.log(1.0 - prob_if_ham)

        prob_if_spam = math.exp(log_prob_if_spam)
        prob_if_ham = math.exp(log_prob_if_ham)
        return prob_if_spam / (prob_if_spam + prob_if_ham)

In [27]:
# Testando o modelo

messages = [Message('spam rules', is_spam=True),
            Message('ham rules', is_spam=False),
            Message('hello ham', is_spam=False)]

model = NaiveBayesClassifier(k=0.5)
model.train(messages)

assert model.tokens == {'spam', 'ham', 'rules', 'hello'}
assert model.spam_messages == 1
assert model.ham_messages == 2
assert model.token_spam_counts == {'spam': 1, 'rules': 1}
assert model.token_ham_counts == {'ham': 2, "rules": 1, "hello": 1}

In [29]:
text = "hello spam"

probs_if_spam = [
    (1 + 0.5) / (1 + 2 * 0.5),      # "spam"  (present)
    1 - (0 + 0.5) / (1 + 2 * 0.5),  # "ham"   (not present)
    1 - (1 + 0.5) / (1 + 2 * 0.5),  # "rules" (not present)
    (0 + 0.5) / (1 + 2 * 0.5)       # "hello" (present)
]

probs_if_ham = [
    (0 + 0.5) / (2 + 2 * 0.5),      # "spam"  (present)
    1 - (2 + 0.5) / (2 + 2 * 0.5),  # "ham"   (not present)
    1 - (1 + 0.5) / (2 + 2 * 0.5),  # "rules" (not present)
    (1 + 0.5) / (2 + 2 * 0.5),      # "hello" (present)
]

p_if_spam = math.exp(sum(math.log(p) for p in probs_if_spam))
p_if_ham = math.exp(sum(math.log(p) for p in probs_if_ham))

assert model.predict(text) == p_if_spam / (p_if_spam + p_if_ham)

In [32]:
from io import BytesIO
import requests
import tarfile

BASE_URL = "https://spamassassin.apache.org/old/publiccorpus"
FILES = ["20021010_easy_ham.tar.bz2", "20021010_hard_ham.tar.bz2", "20021010_spam.tar.bz2"]

# Os dados ficarão aqui, nos subdiretórios /easy_ham, /hard_ham e  /spam
OUTPUT_DIR = "spam_data"

for filename in FILES:
    # Use as solicitações para obter o conteúdo do arquivo em cada url
    content = requests.get(f"{BASE_URL}/{filename}").content
    
    # Encapsule os bytes na memória para usá-los como "arquivo"
    fin = BytesIO(content)

    # E extraia todos os arquivos para o diretório de saída especificado
    with tarfile.open(fileobj=fin, mode="r:bz2") as tf:
        tf.extractall(OUTPUT_DIR)

In [33]:
import glob, re

path = "./datasets/spam_data/*/**"

data: List[Message] = []

# glob.glob retorna todos os nomes de arquivos que correspondem ao caminho curinga
for filename in glob.glob(path):
    is_spam = "ham" not in filename

    # Existem alguns caracteres de lixo nos emails; o erros='ignore' ignora ao inves de gerar uma excessão
    with open(filename, errors='ignore') as email_file:
        for line in email_file:
            if line.startswith('Subject:'):
                subject = line.lstrip("Subject: ")
                data.append(Message(subject, is_spam))
                break

In [36]:
import random
import import_ipynb
from machine_learning import split_data

random.seed(0)
train_messages, test_messages = split_data(data, 0.75)

model = NaiveBayesClassifier()
model.train(train_messages)

In [37]:
from collections import Counter

predictions = [(message, model.predict(message.text)) for message in test_messages]

# Presuma que spam_probability > 0.5 corresponde a variação e spam e conte as combinações de (real is_spam, previsto is_spam)
confusion_matrix = Counter((message.is_spam, spam_probability > 0.5) for message, spam_probability in predictions)

print(confusion_matrix)

Counter({(False, False): 670, (True, True): 86, (True, False): 40, (False, True): 29})


In [38]:
def p_spam_given_token(token: str, model: NaiveBayesClassifier) -> float:
    # Aqui, não recomendo chamar métodos privados, mas é por uma boa causa
    prob_if_spam, prob_if_ham = model._probabilities(token)
    return prob_if_spam / (prob_if_spam + prob_if_ham)

words = sorted(model.tokens, key=lambda t: p_spam_given_token(t, model))

print("spammiest_words", words[-10:])
print("hammiest_words", words[:10])

spammiest_words ['zzzz', 'attn', '95', 'money', 'per', 'clearance', 'rates', 'sale', 'systemworks', 'adv']
hammiest_words ['spambayes', 'users', 'razor', 'zzzzteana', 'sadev', 'apt', 'perl', 'ouch', 'spamassassin', 'bliss']
